## Leaky integrated firing neuron model  
## Few feautures :
### Spatial network, no memory term but exponential decay of cumulative dose

This is a minor change of the Dodds and Watts model  
Implement adaptive rewiring and generate statistics

In [1]:
#importing the required libraries

import networkx as nx
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
# from numba import jit #numba precomplier to make the code faster
import pandas as pd
import copy

# Variables used in the simulation

In [2]:
N = 100
gamma = 5 #decay of cumulative dose per unit time (second)
T = 100 #total number of seconds
dt = 0.1 #seconds
'''number of simulation steps is T/dt'''

'''Dodds watts parameters'''
# p = 10 #rate of dose transfer (per unit time (second))
# p = p*dt #rate of dose transfer per unit simulation step
r = 10
r = r*dt
rho = 10
rho = rho*dt

d_star = 1
dose_quantity = 1
D_ini = 5

'''network parameters'''
circ_rad = 100 #for a network in which each node is connected to other nodes withing a circle of radius circ_rad
nn = 7 #the number of nearest neighbours(i.e. k) in knn
average_degree = 6

'''rewiring parameters'''
p_rew = 5 #rate of rewiring(per unit time(second))
p_rew = p_rew*dt

'rewiring parameters'

# Creating the network. 
### The network outputs coordinates, distance matrix, edge_list and adjacency matrix

In [3]:
'''
obtains the number of nodes N
generates N points
calculates euclidean distance between each pair of points
returns the coordinates of the points and the distance matrix which is N*N
'''
def calc_dist(N) :
    coords = []
    for counter in range(N) :
        coords.append((N*np.random.random(),N*np.random.random()))

    dist_mat = scipy.spatial.distance.cdist(coords,coords)
    return dist_mat,coords

### 1. metric network

In [4]:
'''network = connect to points withing a circle of radius'''
#creates a link between points/nodes which satisfies the conditions of the network
def coupling(dist_mat,circ_rad) :
    edge_list = []
    edge_mask = (dist_mat<circ_rad) & (dist_mat>0)
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(edge_mask))))
#     print(type(np.ndarray.tolist(np.transpose(np.where(edge_mask)))))
    return edge_list

### 2. K nearest neighbour network

In [5]:
'''network = knn'''
def knn(dist_mat,nn,N) :
    near_neigh = np.argsort(dist_mat)
    selec_near_neigh = np.zeros((N,nn))
    selec_near_neigh = near_neigh[:,0:nn+1]

    edge_list = []
    for i in range(N) :
        for j in range(1,nn+1) :
            link = [i,selec_near_neigh[i,j]]
            edge_list.append(link)

    return edge_list

### 3. Random Network

In [6]:
'''random network'''
def rand_network(average_degree,N) :
    z1 = np.random.uniform(size = (N,N))
    E,F = np.meshgrid(np.arange(0,N),np.arange(0,N))
    mask = ((average_degree/N) > z1) & (E!=F)
    adjacency_matrix = np.int64(np.zeros(shape=(N,N)))
    adjacency_matrix[mask] = np.int64(1)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list

# Choose which network you want in the program
### Coupling network of knn network?

In [7]:
'''returns coordinates, distance matrix, edge_list and the adjacency matrix'''

dist_mat,coords = calc_dist(N)
# edge_list = coupling(dist_mat,circ_rad)
# edge_list = knn(dist_mat,nn,N)
edge_list = rand_network(average_degree,N)

adj_mat = np.int64(np.zeros((N,N)))
for i in range(len(edge_list)):
    adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1)

# Function defintions required for the main part of the script

In [8]:
# # @jit(nopython=True)
# '''infecting the left part of space.'''
# def left_part_infec(N) :
#     x_coord = []
#     y_coord = []
#     for j in range(len(coords)) :
#         x_coord.append(coords[j][0])
#         y_coord.append(coords[j][1])
#     x_coord = np.asarray(x_coord)
#     y_coord = np.asarray(y_coord)
#     points = np.asarray(np.where(x_coord<(N/5))) 
#     indi_state[points.T] = 2

#     return indi_state

In [9]:
'''infecting connected nodes'''
def InfectNetworkNeighbors(net,seed_node,init_infected_nodes):
     # if in bulk find one node randomly, and infect its neighbours
    infected_nodes = set()
    candidate_nodes = set()
    explored_nodes = set()

    #pick the seed node
    infected_nodes.add(seed_node)
    explored_nodes.add(seed_node)

    curr_node=seed_node

    #add its neighbors to the list of candidates
    for n in net.neighbors(curr_node):
        candidate_nodes.add(int(n))
    #print( curr_node)
    #print( candidate_nodes)

    #while we need to select more nodes...
    while len(infected_nodes) < init_infected_nodes:

        #if there are candidate nodes, select one of them
        if(len(candidate_nodes) > 0):
            new_node = np.random.choice(list(candidate_nodes),1)[0]
            infected_nodes.add(new_node)
            candidate_nodes.remove(new_node)

        elif len(infected_nodes - explored_nodes) > 0:
            curr_node = np.random.choice(list(infected_nodes -
            explored_nodes),1)[0]
            explored_nodes.add(curr_node)
            for n in set(net.neighbors(curr_node)) - infected_nodes:
                candidate_nodes.add(n)

        else:
            print('Initial node infection step failed')
            return None
    return infected_nodes

In [10]:
''' function to transfer doses'''
# @jit(nopython=True)
def dose(adj_mat,p,partner_state_prev,d) :

    z1 = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    dose_transfer_mask = p > z1
    mod_adj_mat = np.multiply(adj_mat,z1) #modifying adjacency matrix to include the proability of dose transfer
    dose_mask = (p>mod_adj_mat) & (adj_mat!=0) & (partner_state_prev == 2)
    d[dose_mask] = dose_quantity #whenever it is proabable, dose transfer occurs
    
    return d

In [11]:
'''rewiring function which returns a new adjacency matrix'''

def rew(p_rew,adjacency_matrix,indi_state_for_rew_prev) :
    
    z1_rew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    rew_mask_plus = (p_rew > z1_rew)  & (indi_state_for_rew_prev == 1)
    #create links. Dont break any.
    adjacency_matrix[rew_mask_plus] = 1
    rew_mask_minus = (p_rew > z1_rew) & (indi_state_for_rew_prev == 2)
    #break links. Dont create any.
    adjacency_matrix[rew_mask_minus] = 0
    
    np.fill_diagonal(adjacency_matrix,0)
    
    return adjacency_matrix

In [12]:
'''function to update cumulative doses'''
def cumu_dose(d,D_prev,gamma) :
    I = d.sum(axis=1).reshape(N,1)
#     I = 0
    D = (D_prev - (gamma*D_prev*dt)) + I
    return D

In [13]:
'''function to update the states of the nodes of the system'''
def upd_indi_state(D,d_star,indi_state_prev) :
    
    z2 = np.random.uniform(size=(N,1))
    z3 = np.random.uniform(size=(N,1))
    
    indi_state = indi_state_prev
    
    indi_state_mask1 = (D>=d_star) & (indi_state_prev==1)
    indi_state[indi_state_mask1] = 2
    
    indi_state_mask21 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho>=z3)
    indi_state[indi_state_mask21] = 1
    
    indi_state_mask22 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho<z3)
    indi_state[indi_state_mask22] = 3
    
    indi_state_mask23 = (D<d_star) & (indi_state_prev==2) & (r<z2)
    indi_state[indi_state_mask23] = 2
    
    return indi_state

In [14]:
'''function to get/identify the infected nodes and the suceptible nodes'''
def states(indi_state) :
    infec_indi = []
    suscep_indi = []
    infec_indi = np.transpose(np.where(indi_state==2))
    suscep_indi = np.transpose(np.where(indi_state==1))
    return infec_indi,suscep_indi

In [15]:
def networkx_graph(coords,edge_list) :
    G = nx.DiGraph()
    pos = {(i): (coords[i][0],coords[i][1]) for i in range(N)}
    G.add_nodes_from(pos.keys())
    G.add_edges_from(edge_list)
    return G

In [16]:
def edge_from_adj_mat(adj_mat_list,q):
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adj_mat_list[q]==1))))
    return edge_list

# The main part of the script

In [17]:
#infected state time series data frame
#rows indicate parameter (p,start)
#columns indicate
timeseries_infec_frac = pd.DataFrame() 

In [18]:
df_row = 0
for p in range(0,11) : #rate of dose transfer (per unit time (second))
    p = p*dt #rate of dose transfer per unit simulation step
    for start in np.arange(1,11) : #number of nodes to infect initially
        print('(p_rew,p,start) = ', (p_rew,p,start))
        q = 0 #to generate new edge_list from the new adjacency matrix 

        t = np.arange(0,T,dt)

        numpy_array = np.zeros((N,len(t)),dtype = np.int64) #stores the states of the nodes. Rows are nodes and columns are time steps
        D_array = np.zeros((N,len(t))) #stores the cumulative doses. Rows are nodes and columns are time steps

        adj_mat_list = []
        adj_mat_new = copy.deepcopy(adj_mat)
        adj_mat_list.append(adj_mat_new) #list of arrays which shows the time series of the adjacency matrix

        #coords remain the same. Get new edge_list from the latest adjacency matrix
        edge_list = edge_from_adj_mat(adj_mat_list,q)
        G = networkx_graph(coords,edge_list) #networkx graph

        indi_state = np.random.randint(1,2,size=(N,1))
        infected_nodes = None
        while infected_nodes is None : #infecting 'start' number of network neighbours
            infected_nodes = InfectNetworkNeighbors(G,np.random.randint(N),start)
            print(infected_nodes)
        infected_nodes = np.asarray(list(infected_nodes)).reshape(len(infected_nodes),1)
        indi_state[infected_nodes[:,0],0] = 2

        A,B = np.meshgrid(indi_state,indi_state) 
        partner_state = A
        indi_state_for_rew = B

        numpy_array [:,0] = indi_state[:,0]

        indi_state_prev = indi_state
        partner_state_prev = partner_state
        indi_state_for_rew_prev = indi_state_for_rew

        d = np.zeros((len(adj_mat),len(adj_mat[0])))

        D = np.zeros((N,1))
        D[np.where(indi_state==2)] = D_ini
        D_array[:,0] = D[:,0]
        D_prev = D

        infec_frac = np.count_nonzero(indi_state == 2)/N
        timeseries_infec_frac.loc[df_row,0] = infec_frac
        '''the main part of the simulation'''
        counter = 0
        df_col = 1
        for t in np.arange(dt,T,dt) :
            q = q + 1 
            counter = counter + 1
            infec_indi = []
            suscep_indi = []
            d = np.zeros((len(adj_mat),len(adj_mat[0])))
            d = dose(adj_mat,p,partner_state_prev,d)

            adj_mat = rew(p_rew,adj_mat,indi_state_for_rew_prev)
            adj_mat_new = copy.deepcopy(adj_mat)
            adj_mat_list.append(adj_mat_new)

            #coords remain the same. Get new edge_list from the latest adjacency matrix
            edge_list = edge_from_adj_mat(adj_mat_list,q)
            G = networkx_graph(coords,edge_list) #networkx graph

            D = cumu_dose(d,D_prev,gamma)
            D_array[:,counter] = D[:,0]

            indi_state = upd_indi_state(D,d_star,indi_state_prev)
            infec_frac = np.count_nonzero(indi_state == 2)/N
            timeseries_infec_frac.loc[df_row,df_col] = infec_frac

            numpy_array[:,counter] = indi_state[:,0]
            A,B = np.meshgrid(indi_state,indi_state)

            infec_indi, suscep_indi = states(indi_state)

            partner_state = A
            indi_state_for_rew = B
            indi_state_prev = indi_state
            partner_state_prev = partner_state
            indi_state_for_rew_prev = indi_state_for_rew

            D_prev = D
            D = np.zeros((N,1))
            df_col = df_col + 1 

        df_row = df_row + 1 #going to the next row of the df to store timeseries of next (p,start)

(p_rew,p,start) =  (0.0, 0.0, 1)
{26}
(p_rew,p,start) =  (0.0, 0.0, 2)
{10, 58}
(p_rew,p,start) =  (0.0, 0.0, 3)
{1, 77, 87}
(p_rew,p,start) =  (0.0, 0.0, 4)
{57, 49, 93, 31}
(p_rew,p,start) =  (0.0, 0.0, 5)
{37, 7, 42, 74, 28}
(p_rew,p,start) =  (0.0, 0.0, 6)
{0, 2, 9, 18, 93, 61}
(p_rew,p,start) =  (0.0, 0.0, 7)
{97, 1, 12, 15, 16, 49, 84}
(p_rew,p,start) =  (0.0, 0.0, 8)
{96, 1, 64, 41, 44, 77, 48, 29}
(p_rew,p,start) =  (0.0, 0.0, 9)
{6, 71, 40, 73, 18, 55, 62, 25, 30}
(p_rew,p,start) =  (0.0, 0.0, 10)
{97, 1, 69, 7, 41, 11, 44, 46, 79, 91}
(p_rew,p,start) =  (0.0, 0.1, 1)
{72}
(p_rew,p,start) =  (0.0, 0.1, 2)
{8, 42}
(p_rew,p,start) =  (0.0, 0.1, 3)
{65, 2, 59}
(p_rew,p,start) =  (0.0, 0.1, 4)
{64, 90, 42, 38}
(p_rew,p,start) =  (0.0, 0.1, 5)
{70, 40, 85, 28, 62}
(p_rew,p,start) =  (0.0, 0.1, 6)
{68, 73, 12, 81, 84, 61}
(p_rew,p,start) =  (0.0, 0.1, 7)
{35, 3, 76, 46, 16, 83, 27}
(p_rew,p,start) =  (0.0, 0.1, 8)
{5, 9, 43, 19, 86, 88, 89, 93}
(p_rew,p,start) =  (0.0, 0.1, 9)
{97, 

(p_rew,p,start) =  (0.1, 0.30000000000000004, 1)
{51}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 2)
{57, 7}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 3)
{16, 25, 59}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 4)
{17, 91, 92, 71}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 5)
{33, 67, 50, 84, 85}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 6)
{32, 77, 81, 87, 89, 29}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 7)
{9, 60, 79, 86, 26, 27, 28}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 8)
{64, 69, 71, 10, 19, 85, 24, 63}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 9)
{34, 4, 9, 44, 48, 18, 50, 88, 90}
(p_rew,p,start) =  (0.1, 0.30000000000000004, 10)
{64, 65, 2, 5, 7, 72, 43, 17, 87, 61}
(p_rew,p,start) =  (0.1, 0.4, 1)
{48}
(p_rew,p,start) =  (0.1, 0.4, 2)
{77, 7}
(p_rew,p,start) =  (0.1, 0.4, 3)
{75, 60, 38}
(p_rew,p,start) =  (0.1, 0.4, 4)
{3, 85, 61, 63}
(p_rew,p,start) =  (0.1, 0.4, 5)
{1, 99, 5, 71, 55}
(p_rew,p,start) =  (0.1, 0.4, 6)
{33, 36, 13, 19, 53, 89}
(p

(p_rew,p,start) =  (0.2, 0.5, 4)
{26, 69, 13, 87}
(p_rew,p,start) =  (0.2, 0.5, 5)
{64, 0, 71, 53, 23}
(p_rew,p,start) =  (0.2, 0.5, 6)
{70, 71, 40, 77, 29, 30}
(p_rew,p,start) =  (0.2, 0.5, 7)
{68, 7, 45, 85, 21, 89, 29}
(p_rew,p,start) =  (0.2, 0.5, 8)
{96, 36, 49, 84, 52, 87, 26, 28}
(p_rew,p,start) =  (0.2, 0.5, 9)
{5, 6, 71, 41, 15, 81, 52, 86, 94}
(p_rew,p,start) =  (0.2, 0.5, 10)
{96, 35, 36, 78, 79, 50, 83, 20, 31, 63}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 1)
{58}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 2)
{76, 21}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 3)
{33, 36, 93}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 4)
{89, 42, 60, 38}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 5)
{36, 15, 53, 54, 94}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 6)
Initial node infection step failed
None
{6, 75, 77, 15, 57, 61}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 7)
{0, 45, 51, 52, 53, 56, 89}
(p_rew,p,start) =  (0.2, 0.6000000000000001, 8)
{65, 68, 44, 78, 87, 25, 58,

(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 1)
{8}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 2)
{62, 7}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 3)
{97, 26, 30}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 4)
{34, 35, 28, 36}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 5)
{1, 97, 22, 90, 92}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 6)
{96, 6, 59, 89, 27, 62}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 7)
Initial node infection step failed
None
{33, 38, 39, 13, 52, 85, 58}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 8)
{4, 11, 12, 13, 14, 48, 82, 26}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 9)
{96, 70, 38, 8, 44, 78, 46, 55, 26}
(p_rew,p,start) =  (0.30000000000000004, 0.6000000000000001, 10)
{65, 36, 41, 44, 81, 51, 21, 56, 94, 31}
(p_rew,p,start) =  (0.30000000000000004, 0.7000000000000001, 1)
{91}
(p_rew,p,start) =  (0.3000

(p_rew,p,start) =  (0.4, 0.6000000000000001, 8)
{66, 98, 39, 75, 48, 81, 19, 53}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 9)
Initial node infection step failed
None
{69, 71, 75, 14, 47, 85, 22, 29, 63}
(p_rew,p,start) =  (0.4, 0.6000000000000001, 10)
{99, 9, 75, 44, 77, 49, 82, 89, 58, 95}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 1)
{34}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 2)
{64, 84}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 3)
Initial node infection step failed
None
{51, 42, 75}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 4)
{73, 92, 70, 9}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 5)
{70, 76, 79, 87, 55}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 6)
{96, 45, 18, 83, 87, 31}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 7)
Initial node infection step failed
None
Initial node infection step failed
None
{68, 11, 13, 84, 57, 25, 63}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 8)
{32, 1, 43, 52, 53, 88, 57, 61}
(p_rew,p,start) =  (0.4, 0.7000000000000001, 9)
Ini

...

(p_rew,p,start) =  (0.9, 1.0, 3)
Initial node infection step failed
None
{72, 69, 38}
(p_rew,p,start) =  (0.9, 1.0, 4)
Initial node infection step failed
None
{88, 81, 51, 37}
(p_rew,p,start) =  (0.9, 1.0, 5)
Initial node infection step failed
None
{2, 36, 48, 20, 21}
(p_rew,p,start) =  (0.9, 1.0, 6)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{9, 43, 77, 49, 25, 93}
(p_rew,p,start) =  (0.9, 1.0, 7)
{64, 66, 38, 10, 14, 25, 92}
(p_rew,p,start) =  (0.9, 1.0, 8)
Initial node infection step failed
None
{99, 39, 77, 49, 20, 30, 90, 62}
(p_rew,p,start) =  (0.9, 1.0, 9)
{89, 36, 7, 81, 54, 22, 25, 93, 30}
(p_rew,p,start) =  (0.9, 1.0, 10)
Initial node infection step failed
None
Initial node infection step failed
None
{96, 97, 1, 68, 71, 79, 19, 89, 90, 94}
(p_rew,p,start) =  (1.0, 0.0, 1)
{26}
(p_rew,p,start) =  (1.0, 0.0, 2)
{21, 5}
(p_rew,p,start) =  (1.0, 0.0, 3)
{96, 17, 11}
(p_rew,p,start) =  (1.0, 0.0, 4)
{16, 

(p_rew,p,start) =  (1.0, 0.4, 3)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{74, 35, 39}
(p_rew,p,start) =  (1.0, 0.4, 4)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
I

(p_rew,p,start) =  (1.0, 0.5, 1)
{72}
(p_rew,p,start) =  (1.0, 0.5, 2)
Initial node infection step failed
None
Initial node infection step failed
None
{72, 31}
(p_rew,p,start) =  (1.0, 0.5, 3)
{50, 4, 20}
(p_rew,p,start) =  (1.0, 0.5, 4)
Initial node infection step failed
None
Initial node infection step failed
None
{3, 34, 75, 86}
(p_rew,p,start) =  (1.0, 0.5, 5)
{64, 66, 37, 47, 83}
(p_rew,p,start) =  (1.0, 0.5, 6)
Initial node infection step failed
None
{64, 38, 78, 86, 87, 61}
(p_rew,p,start) =  (1.0, 0.5, 7)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{1, 75, 76, 16, 81, 55, 60}
(p_rew,p,start) =  (1.0, 0.5, 8)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infecti

(p_rew,p,start) =  (1.0, 0.8, 7)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{97, 34, 66, 69, 17, 90, 62}
(p_rew,p,start) =  (1.0, 0.8, 8)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection st

(p_rew,p,start) =  (1.0, 0.8, 10)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
{65, 99, 4, 7, 80, 18, 51, 22, 62, 95}
(p_rew,p,start) =  (1.0, 0.9, 1)
{57}
(p_rew,p,start) =  (1.0, 0.9, 2)
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node infection step failed
None
Initial node inf

In [19]:
# '''time series plot of a fixed start but varying p'''
# a = 4 #this is the value of p-1
# for i in np.arange(0,110,10) : 
#     print(a+i)
#     timeseries_infec_frac.loc[a+i,:].plot()
# plt.ylim(-0.1,1.1)
# # plt.xlim(0,10)
# plt.legend(loc=(1,1))
# plt.show()

In [20]:
# plt.clf()
# '''time series plot of a fixed p but varying start'''
# a = 40 # this is the value of p
# for i in range(0,11) :
#     timeseries_infec_frac.loc[a+i,:].plot()
# plt.ylim(-0.1,1.1)
# # plt.ylim(-0.01,0.11)
# # plt.xlim(0,2.5)
# plt.legend(loc=(1,1))
# plt.show()

In [21]:
# start_len = 10
# p_len = 11
# X,Y = np.meshgrid([i*0.01 for i in range(11,21)],[i*0.1 for i in range(0,11)])
# print('X = ', X)
# print('Y = ', Y)
# phi = 999*np.ones((p_len,start_len))
# counter = 0
# last_col = np.int64((T/dt)-1)
# for i in range(p_len) :
#     for j in range(start_len) :
#         phi[i,j] = timeseries_infec_frac.loc[counter,(last_col-100)+1:last_col].mean()
#         counter = counter + 1
# # print(counter)
# print(phi)
# fig = plt.gcf()
# fig.set_size_inches(10, 9)
# plt.scatter(X, Y,s=500,c=phi,vmin=0,vmax=1)
# plt.colorbar()
# plt.xlabel(r'$\phi_{ini}$', fontsize = 20)
# plt.ylabel(r'$p\times dt$', fontsize = 20)
# plt.xlim(0.10,0.21)
# plt.title(r'$\phi^*,p_{rew}=0.5,av_{deg}=9,d=1,d^*=3,\gamma=5,T=N=100,dt=0.1,r=1$',fontsize = 15)
# plt.savefig('static_avdeg_9_ini_0_11.svg',format='svg')
# plt.show()

In [22]:
# filename = 'file_avdeg_'+str(average_degree)+'_ini_'+str(np.int64(X[0,0]*N))+'_'+str(np.int64(X[p_len-1,start_len-1]*N))+'.h5'
# key 
# timeseries_infec_frac.to_hdf(filename, key='timeseries_infec_frac', mode='w')

In [28]:
# import h5py
# f = h5py.File('file_avdeg_6_ini_0_11.h5', 'r')
# [key for key in f.keys()]
# pd.read_hdf('file_avdeg_6_ini_0_11_firstattemp.h5','p_rew_1')

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.02,0.02,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.03,0.03,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.04,0.04,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.05,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.06,0.06,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.07,0.07,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.08,0.08,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.09,0.09,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.10,0.10,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
